In [11]:
from datetime import datetime, timedelta

class Material:
    def __init__(self, id_material, titulo, stock=1):
        self._id_material = id_material
        self._titulo = titulo
        self._stock = stock  # stock disponible

    def mostrar_info(self):
        return f"Material {self._id_material}: {self._titulo} - Stock: {self._stock}"

    def prestar(self):
        if self._stock > 0:
            self._stock -= 1
            return True
        return False

    def devolver(self):
        self._stock += 1

    def get_plazo(self):
        # Por defecto 14 días y multa 300, se sobreescribe en subclases
        return 14

    def get_multa_por_dia(self):
        return 300

class Libro(Material):
    def __init__(self, id_material, titulo, autor, stock=1):
        super().__init__(id_material, titulo, stock)
        self._autor = autor

    def mostrar_info(self):
        base_info = super().mostrar_info()
        return f"{base_info} (Libro de {self._autor})"

    def get_plazo(self):
        return 14

    def get_multa_por_dia(self):
        return 300

class Revista(Material):
    def __init__(self, id_material, titulo, stock=1):
        super().__init__(id_material, titulo, stock)

    def mostrar_info(self):
        base_info = super().mostrar_info()
        return f"{base_info} (Revista)"

    def get_plazo(self):
        return 7

    def get_multa_por_dia(self):
        return 200

class Usuario:
    def __init__(self, id_usuario, nombre):
        self._id_usuario = id_usuario
        self._nombre = nombre

    def mostrar_info(self):
        return f"Usuario {self._id_usuario}: {self._nombre}"

class Prestamo:
    def __init__(self, usuario, material):
        self._usuario = usuario
        self._material = material
        self._fecha_prestamo = datetime.now()
        plazo = material.get_plazo()
        self._fecha_devolucion = self._fecha_prestamo + timedelta(days=plazo)
        self._devuelto = False
        self._fecha_devolucion_real = None
        self._penalizacion = 0

    def devolver(self):
        self._devuelto = True
        self._fecha_devolucion_real = datetime.now()
        dias_atraso = (self._fecha_devolucion_real - self._fecha_devolucion).days
        dias_atraso = max(0, dias_atraso)
        multa_dia = self._material.get_multa_por_dia()
        self._penalizacion = dias_atraso * multa_dia
        self._material.devolver()
        return self._penalizacion

    def mostrar_info(self):
        estado = "Devuelto" if self._devuelto else "Activo"
        info = (f"Préstamo: {self._material._titulo} a {self._usuario._nombre} "
                f"desde {self._fecha_prestamo.strftime('%Y-%m-%d')} "
                f"hasta {self._fecha_devolucion.strftime('%Y-%m-%d')} - {estado}")
        if self._devuelto:
            info += f" - Penalización: ${self._penalizacion}"
        return info

class Biblioteca:
    def __init__(self):
        self._usuarios = {}
        self._materiales = {}
        self._prestamos = []

    def agregar_usuario(self, id_usuario, nombre):
        if id_usuario in self._usuarios:
            print("Usuario ya existe.")
            return
        self._usuarios[id_usuario] = Usuario(id_usuario, nombre)
        print("Usuario agregado.")

    def agregar_libro(self, id_material, titulo, autor, stock=1):
        if id_material in self._materiales:
            print("Material ya existe.")
            return
        self._materiales[id_material] = Libro(id_material, titulo, autor, stock)
        print("Libro agregado.")

    def agregar_revista(self, id_material, titulo, stock=1):
        if id_material in self._materiales:
            print("Material ya existe.")
            return
        self._materiales[id_material] = Revista(id_material, titulo, stock)
        print("Revista agregada.")

    def prestar_material(self, id_usuario, id_material):
        usuario = self._usuarios.get(id_usuario)
        material = self._materiales.get(id_material)
        if not usuario:
            print("Usuario no encontrado.")
            return
        if not material:
            print("Material no encontrado.")
            return
        # Verificar si usuario ya tiene préstamo activo del mismo material
        for p in self._prestamos:
            if (p._usuario._id_usuario == id_usuario and
                p._material._id_material == id_material and
                not p._devuelto):
                print("El usuario ya tiene un préstamo activo de este material.")
                return
        if material.prestar():
            prestamo = Prestamo(usuario, material)
            self._prestamos.append(prestamo)
            print("Préstamo realizado.")
        else:
            print("Material no disponible.")

    def devolver_material(self, id_usuario, id_material):
        # Buscar préstamo activo para ese usuario y material
        prestamo_encontrado = None
        for p in self._prestamos:
            if (p._usuario._id_usuario == id_usuario and
                p._material._id_material == id_material and
                not p._devuelto):
                prestamo_encontrado = p
                break
        if not prestamo_encontrado:
            print("No se encontró préstamo activo para ese usuario y material.")
            return
        penalizacion = prestamo_encontrado.devolver()
        print(f"Material devuelto. Penalización: ${penalizacion}")

    def listar_usuarios(self):
        for u in self._usuarios.values():
            print(u.mostrar_info())

    def listar_materiales(self):
        for m in self._materiales.values():
            print(m.mostrar_info())

    def listar_prestamos(self):
        for p in self._prestamos:
            print(p.mostrar_info())

def menu():
    biblioteca = Biblioteca()
    while True:
        print("\n--- Menú Biblioteca ---")
        print("1. Agregar usuario")
        print("2. Agregar libro")
        print("3. Agregar revista")
        print("4. Prestar material")
        print("5. Devolver material")
        print("6. Listar usuarios")
        print("7. Listar materiales")
        print("8. Listar préstamos")
        print("0. Salir")
        opcion = input("Seleccione una opción: ")

        if opcion == "1":
            id_u = input("ID usuario: ")
            nombre = input("Nombre: ")
            biblioteca.agregar_usuario(id_u, nombre)
        elif opcion == "2":
            id_m = input("ID material: ")
            titulo = input("Título: ")
            autor = input("Autor: ")
            stock = int(input("Stock: "))
            biblioteca.agregar_libro(id_m, titulo, autor, stock)
        elif opcion == "3":
            id_m = input("ID material: ")
            titulo = input("Título: ")
            stock = int(input("Stock: "))
            biblioteca.agregar_revista(id_m, titulo, stock)
        elif opcion == "4":
            id_u = input("ID usuario: ")
            id_m = input("ID material: ")
            biblioteca.prestar_material(id_u, id_m)
        elif opcion == "5":
            id_u = input("ID usuario: ")
            id_m = input("ID material: ")
            biblioteca.devolver_material(id_u, id_m)
        elif opcion == "6":
            biblioteca.listar_usuarios()
        elif opcion == "7":
            biblioteca.listar_materiales()
        elif opcion == "8":
            biblioteca.listar_prestamos()
        elif opcion == "0":
            print("Saliendo...")
            break
        else:
            print("Opción inválida.")

if __name__ == "__main__":
    menu()


--- Menú Biblioteca ---
1. Agregar usuario
2. Agregar libro
3. Agregar revista
4. Prestar material
5. Devolver material
6. Listar usuarios
7. Listar materiales
8. Listar préstamos
0. Salir
Seleccione una opción: 1
ID usuario: 12345
Nombre: Santiago
Usuario agregado.

--- Menú Biblioteca ---
1. Agregar usuario
2. Agregar libro
3. Agregar revista
4. Prestar material
5. Devolver material
6. Listar usuarios
7. Listar materiales
8. Listar préstamos
0. Salir
Seleccione una opción: 6
Usuario 12345: Santiago

--- Menú Biblioteca ---
1. Agregar usuario
2. Agregar libro
3. Agregar revista
4. Prestar material
5. Devolver material
6. Listar usuarios
7. Listar materiales
8. Listar préstamos
0. Salir
Seleccione una opción: 2
ID material: 123
Título: La Monalisa
Autor: Leonardo Davinci
Stock: 5
Libro agregado.

--- Menú Biblioteca ---
1. Agregar usuario
2. Agregar libro
3. Agregar revista
4. Prestar material
5. Devolver material
6. Listar usuarios
7. Listar materiales
8. Listar préstamos
0. Salir
Se

KeyboardInterrupt: Interrupted by user